In [2]:
# make the code as Python 3 compatible as possible
from __future__ import print_function, division
import pypsa
import numpy as np

In [3]:
network = pypsa.Network()
csv_folder = "data\minimal_example"

#add buses
network.import_from_csv_folder(csv_folder_name = csv_folder)




#for i in range(3):
#    network.add("Bus","My bus {}".format(i))
#
print(network.buses)


AssertionError: Directory data\minimal_example does not exist.

In [3]:
#add three lines in a ring
for i in range(3):
    network.add("Line","My line {}".format(i),
                bus0="My bus {}".format(i),
                bus1="My bus {}".format((i+1)%3),
                x=0.0001,
                s_nom=60)
    


#add a generator at bus 0
network.add("Generator","My gen 0",
            bus="My bus 0",
            p_nom=100,
            marginal_cost=50)

#add a generator at bus 1
network.add("Generator","My gen 1",
            bus="My bus 1",
            p_nom=100,
            marginal_cost=25)

#add a load at bus 2
network.add("Load","My load",
            bus="My bus 2",
            p_set=100)

In [4]:
print(network.lines)
print(network.generators)
print(network.generators.p_set)

attribute      bus0      bus1 type       x    r    g    b  s_nom  \
My line 0  My bus 0  My bus 1       0.0001  0.0  0.0  0.0   60.0   
My line 1  My bus 1  My bus 2       0.0001  0.0  0.0  0.0   60.0   
My line 2  My bus 2  My bus 0       0.0001  0.0  0.0  0.0   60.0   

attribute  s_nom_extendable  s_nom_min  ...  v_ang_min  v_ang_max  \
My line 0             False        0.0  ...       -inf        inf   
My line 1             False        0.0  ...       -inf        inf   
My line 2             False        0.0  ...       -inf        inf   

attribute  sub_network  x_pu  r_pu  g_pu  b_pu  x_pu_eff r_pu_eff  s_nom_opt  
My line 0                0.0   0.0   0.0   0.0       0.0      0.0        0.0  
My line 1                0.0   0.0   0.0   0.0       0.0      0.0        0.0  
My line 2                0.0   0.0   0.0   0.0       0.0      0.0        0.0  

[3 rows x 26 columns]
attribute       bus control type  p_nom  p_nom_extendable  p_nom_min  \
My gen 0   My bus 0      PQ       100.0

In [5]:
#Do a linear OPF

def my_f(network,snapshots):
    print(snapshots)

network.lopf(solver_name="gurobi", formulation ="angles", extra_functionality = my_f)

#Cheap generator 1 cannot be fully dispatched because of network constraints,
#so expensive generator 0 also has to dispatch
print(network.generators_t.p)

#network flows
print(network.lines_t.p0)

#Line 1 is congested
print(abs(network.lines_t.p0)/network.lines.s_nom)

#Power flows towards lower voltage angles
print(network.buses_t.v_ang*180/np.pi)

#In linear approximation, all voltage magnitudes are nominal, i.e. 1 per unit
print(network.buses_t.v_mag_pu)

#At bus 2 the price is set above any marginal generation costs in the model, because to dispatch to
#it from expensive generator 0, also some dispatch from cheap generator 1 has to be substituted from generator0
#to avoid overloading line 1.
print(network.buses_t.marginal_price)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using gurobi


Index(['now'], dtype='object')


INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x9
  Lower bound: 3000.0
  Upper bound: 3000.0
  Number of objectives: 1
  Number of constraints: 14
  Number of variables: 9
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 9
  Number of nonzeros: 25
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subjec